In [2]:
import pandas as pd 
import os
from sklearn import preprocessing 
from collections import deque
import random 
import numpy as np
import time 

In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, CuDNNLSTM, BatchNormalization, RNN
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import ModelCheckpoint

In [4]:
SEQ_LEN = (range(10,10,20))  # how long of a preceeding sequence to collect for RNN
FUTURE_PERIOD_PREDICT = range(5,20,5)  # how far into the future are we trying to predict?
RATIO_TO_PREDICT = "Dow_Jones"
EPOCHS = 10
BATCH_SIZE = 64

def classify(current, future):
    if float(future) > float(current):  # if the future price is higher than the current, that's a buy, or a 1
        return 1
    else:  # otherwise... it's a 0!
        return 0

def preprocess_df(df):
    df = df.drop("future", 1)  # don't need this anymore.

    for col in df.columns:  # go through all of the columns
        if col != "target":  # normalize all ... except for the target itself!
            df[col] = df[col].pct_change()  # pct change "normalizes" the different currencies (each crypto coin has vastly diff values, we're really more interested in the other coin's movements)
            df.dropna(inplace=True)  # remove the nas created by pct_change
            df[col] = preprocessing.scale(df[col].values)  # scale between 0 and 1.

    df.dropna(inplace=True)  # cleanup again... jic. Those nasty NaNs love to creep in.

    sequential_data = []  # this is a list that will CONTAIN the sequences
    prev_days = deque(maxlen=SEQ_LEN)  # These will be our actual sequences. They are made with deque, which keeps the maximum length by popping out older values as new ones come in

    for i in df.values:  # iterate over the values
        prev_days.append([n for n in i[:-1]])  # store all but the target
        if len(prev_days) == SEQ_LEN:  # make sure we have 60 sequences!
            sequential_data.append([np.array(prev_days), i[-1]])  # append those bad boys!

    random.shuffle(sequential_data)  # shuffle for good measure.

    buys = []
    sells = []
    for seq, target in sequential_data:
        if target == 1:
            buys.append([seq, target])
        elif target == 0:
                sells.append([seq, target])
    random.shuffle(buys)
    random.shuffle(sells)
    
    lower = min(len(buys), len(sells))
    buys = buys[:lower]
    sells = sells[:lower]
    
    sequential_data = buys + sells 
    random.shuffle(sequential_data)
    
    X = []
    y = []
    for seq, target in sequential_data:
        X.append(seq)
        y.append(target)
        
    return np.array(X), y


main_df = pd.DataFrame() # begin empty

ratios = ["Dow_Jones", "Nas_Daq", "S&P500"]  # the 3 ratios we want to consider
for ratio in ratios:  # begin iteration
    print(ratio)
    dataset = f'Indexes_data/{ratio}.csv'  # get the full path to the file.
    df = pd.read_csv(dataset)  # read in specific file

    # rename volume and close to include the ticker so we can still which close/volume is which:
    df.rename(columns={"Close": f"{ratio}_close", "Volume": f"{ratio}_volume"}, inplace=True)

    df.set_index("Date", inplace=True)  # set time as index so we can join them on this shared time
    df = df[[f"{ratio}_close", f"{ratio}_volume"]]  # ignore the other columns besides price and volume

    if len(main_df)==0:  # if the dataframe is empty
        main_df = df  # then it's just the current df
    else:  # otherwise, join this data to the main one
        main_df = main_df.join(df)

main_df.fillna(method="ffill", inplace=True)  # if there are gaps in data, use previously known values
main_df.dropna(inplace=True)
print(main_df.head())  # how did we do??

Dow_Jones
Nas_Daq
S&P500
            Dow_Jones_close  Dow_Jones_volume  Nas_Daq_close  Nas_Daq_volume  \
Date                                                                           
1985-10-01      1340.949951          15600000     112.139999     153160000.0   
1985-10-02      1333.670044          12580000     110.824997     164640000.0   
1985-10-03      1333.109985          10700000     110.870003     147300000.0   
1985-10-04      1328.739990           8830000     110.074997     147900000.0   
1985-10-07      1324.369995           9300000     108.199997     128640000.0   

            S&P500_close  S&P500_volume  
Date                                     
1985-10-01    185.070007    130200000.0  
1985-10-02    184.059998    147300000.0  
1985-10-03    184.360001    127500000.0  
1985-10-04    183.220001    101200000.0  
1985-10-07    181.869995     95550000.0  


In [5]:
main_df['future'] = main_df[f'{RATIO_TO_PREDICT}_close'].shift(-FUTURE_PERIOD_PREDICT)
main_df['target'] = list(map(classify, main_df[f'{RATIO_TO_PREDICT}_close'], main_df['future']))

#print(main_df.head())

times = sorted(main_df.index.values)  # get the times
last_5pct = sorted(main_df.index.values)[-int(0.05*len(times))]  # get the last 5% of the times

validation_main_df = main_df[(main_df.index >= last_5pct)]  # make the validation data where the index is in the last 5%
main_df = main_df[(main_df.index < last_5pct)]  # now the main_df is all the data up to the last 5%
validation_main_df = validation_main_df.dropna()

train_x, train_y = preprocess_df(main_df)
validation_x, validation_y = preprocess_df(validation_main_df)

print(f"train data: {len(train_x)} validation: {len(validation_x)}")
print(f"Dont buys: {train_y.count(0)}, buys: {train_y.count(1)}")
print(f"VALIDATION Dont buys: {validation_y.count(0)}, buys: {validation_y.count(1)}")

TypeError: bad operand type for unary -: 'range'

In [ ]:
for i in SEQ_LEN:
    for k in FUTURE_PERIOD_PREDICT:
        NAME = f"{SEQ_LEN}-SEQ-{FUTURE_PERIOD_PREDICT}-PRED-{int(time.time())}"
        # Model 
        model = Sequential()
        model.add(LSTM(32, input_shape=(train_x.shape[1:]), return_sequences=True))
        model.add(Dropout(0.2))
        model.add(BatchNormalization())  #normalizes activation outputs, same reason you want to normalize your input data.

        model.add(LSTM(32, return_sequences=True))
        model.add(Dropout(0.1))
        model.add(BatchNormalization())

        model.add(LSTM(32))
        model.add(Dropout(0.2))
        model.add(BatchNormalization())

        model.add(Dense(32, activation='relu'))
        model.add(Dropout(0.2))

        model.add(Dense(2, activation='softmax'))
        
        opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)

        # Compile model
        model.compile(
            loss='sparse_categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])
        tensorboard = TensorBoard(log_dir="logs_2\{}".format(NAME))
        
        filepath = "RNN_Final-{epoch:02d}-{val_acc:.3f}"  # unique file name that will include the epoch and the validation acc for that epoch
        checkpoint = ModelCheckpoint("models_1\{}.model".format(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')) # saves only the best ones
        
        # Train model
        history = model.fit(
            train_x, train_y,
            batch_size=BATCH_SIZE,
            epochs=EPOCHS,
            validation_data=(validation_x, validation_y),
            callbacks=[tensorboard, checkpoint],
        )